# Lab 0: Generate Teacher Logits

This notebook generates teacher model logits for knowledge distillation. The teacher model (Qwen3-30B-A3B) processes a dataset and outputs logits that will be used to train a smaller student model.

## Import Dependencies

Import required libraries for model inference, tokenization, and Neuron-specific configurations.

In [1]:
import torch
import json
import argparse

from transformers import AutoTokenizer, GenerationConfig
from neuronx_distributed_inference.models.config import MoENeuronConfig, OnDeviceSamplingConfig
from neuronx_distributed_inference.models.qwen3_moe.modeling_qwen3_moe import Qwen3MoeInferenceConfig, NeuronQwen3MoeForCausalLM
from neuronx_distributed_inference.utils.hf_adapter import HuggingFaceGenerationAdapter, load_pretrained_config

torch.manual_seed(0)

/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise.py:68: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  component, error = import_nki(config)
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise.py:68: Depreca

## Configuration

Set model paths and file locations. The teacher model will be compiled and saved to the traced model path for efficient inference on AWS Neuron.

In [8]:
model_path = "Qwen/Qwen3-30B-A3B"
traced_model_path = "/home/ubuntu/traced_model/Qwen3-30B-A3B/"
dataset_file = "data/dataset.txt"
output_file = "output.json"

## Create Conversation Template

Define a function to format input text as a conversation for the sentiment classification task.

In [3]:
def create_conversation(sample):
    system_message = (
        "You are a sentiment classifier. You take input strings and return the sentiment of POSITIVE, NEGATIVE, or NEUTRAL. Only return the sentiment."
    )
    return [
        {
            "role": "system",
            "content": system_message,
        },
        {
            "role": "user",
            "content": sample
        },
    ]

## Initialize Model Configuration

Configure the Neuron-specific settings for the Qwen3 MoE model:
- Tensor parallelism degree: 8 (distributes model across 8 NeuronCores)
- Enable logit output for distillation
- Set sampling parameters for generation

In [4]:
generation_config = GenerationConfig.from_pretrained(model_path)

neuron_config = MoENeuronConfig(
    tp_degree=8,
    batch_size=1,
    max_context_length=128,
    seq_len=1024,
    on_device_sampling_config=OnDeviceSamplingConfig(do_sample=True, temperature=0.6, top_k=20, top_p=0.95),
    enable_bucketing=False,
    flash_decoding_enabled=False,
    output_scores=True,
    output_logits=True
)

config = Qwen3MoeInferenceConfig(
    neuron_config,
    load_config=load_pretrained_config(model_path),
)

tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right")
tokenizer.pad_token = tokenizer.eos_token

## Compile and Save Model

Compile the model for AWS Neuron hardware. This step converts the model to a Neuron-optimized format and saves it for reuse.

In [5]:
print("\nCompiling and saving model...")
model = NeuronQwen3MoeForCausalLM(model_path, config)
model.compile(traced_model_path)
tokenizer.save_pretrained(traced_model_path)

INFO:Neuron:Saving the neuron_config to /home/ubuntu/traced_model/Qwen3-30B-A3B/
INFO:Neuron:Generating HLOs for the following models: ['context_encoding_model', 'token_generation_model']



Compiling and saving model...
[2025-10-23 18:41:51.532: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 8
[2025-10-23 18:41:51.533: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-10-23 18:41:51.534: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-10-23 18:41:51.535: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-10-23 18:41:51.536: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 8
[2025-10-23 18:41:51.536: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x754b1f3b9cf0>, 'Ascending Ring PG Group')>
[2025-10-23 18:41:51.537: I neuronx_distributed

INFO:Neuron:Generating 1 hlos for key: context_encoding_model
INFO:Neuron:Started loading module context_encoding_model
INFO:Neuron:Finished loading module context_encoding_model in 0.6785175800323486 seconds
INFO:Neuron:generating HLO: context_encoding_model, input example shape = torch.Size([1, 128])


[2025-10-23 18:41:55.678: W neuronx_distributed/modules/moe/expert_mlps_v2.py:765] T 128 not divisible by block_size 512, cannot use index calc kernel.
[2025-10-23 18:41:55.760: W neuronx_distributed/modules/moe/expert_mlps_v2.py:765] T 128 not divisible by block_size 512, cannot use index calc kernel.
[2025-10-23 18:41:55.772: W neuronx_distributed/modules/moe/expert_mlps_v2.py:765] T 128 not divisible by block_size 512, cannot use index calc kernel.
[2025-10-23 18:41:55.783: W neuronx_distributed/modules/moe/expert_mlps_v2.py:765] T 128 not divisible by block_size 512, cannot use index calc kernel.
[2025-10-23 18:41:55.795: W neuronx_distributed/modules/moe/expert_mlps_v2.py:765] T 128 not divisible by block_size 512, cannot use index calc kernel.
[2025-10-23 18:41:55.807: W neuronx_distributed/modules/moe/expert_mlps_v2.py:765] T 128 not divisible by block_size 512, cannot use index calc kernel.
[2025-10-23 18:41:55.818: W neuronx_distributed/modules/moe/expert_mlps_v2.py:765] T 128

/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:507: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/torch/autograd/function.py:575: DeprecationWarning: Block dimension is deprecated. The leading dimension of SBUF tensor must be partition dimension
  return super().apply(*args, **kwargs)  # type: ignore[misc]
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/torch/autograd/function.py:575: DeprecationWarning: Block dimension is deprecated. The leading dimension of SBUF tensor must be partition dimension
  return super().apply(*args, **kwargs)  # type: ignore[misc]
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/torch/autograd/function.py:575: DeprecationWarning: Block dimension is deprecated. The leading dimension of

...

..

INFO:Neuron:Done preparing weight layout transformation


Completed run_backend_driver.

Compiler status PASS


INFO:Neuron:Finished building model in 55.69548773765564 seconds
INFO:Neuron:SKIPPING pre-sharding the checkpoints. The checkpoints will be sharded during load time.


('/home/ubuntu/traced_model/Qwen3-30B-A3B/tokenizer_config.json',
 '/home/ubuntu/traced_model/Qwen3-30B-A3B/special_tokens_map.json',
 '/home/ubuntu/traced_model/Qwen3-30B-A3B/chat_template.jinja',
 '/home/ubuntu/traced_model/Qwen3-30B-A3B/vocab.json',
 '/home/ubuntu/traced_model/Qwen3-30B-A3B/merges.txt',
 '/home/ubuntu/traced_model/Qwen3-30B-A3B/added_tokens.json',
 '/home/ubuntu/traced_model/Qwen3-30B-A3B/tokenizer.json')

## Load Compiled Model

Load the compiled model from disk for inference.

In [6]:
model = NeuronQwen3MoeForCausalLM(traced_model_path)
model.load(traced_model_path)
tokenizer = AutoTokenizer.from_pretrained(traced_model_path)

INFO:Neuron:Sharding weights on load...
INFO:Neuron:Sharding Weights for ranks: 0...7


[2025-10-23 18:43:14.892: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 8
[2025-10-23 18:43:14.893: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-10-23 18:43:14.894: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-10-23 18:43:14.895: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-10-23 18:43:14.896: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 8
[2025-10-23 18:43:14.897: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x754b1f3b9cf0>, 'Ascending Ring PG Group')>
[2025-10-23 18:43:14.899: I neuronx_distributed/parallel_layers/parallel_state

Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed_inference/models/application_base.py:552: UserWarning: Found torch.float32 weights in checkpoint: lm_head.weight. Will convert to torch.bfloat16
  warnings.warn(
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed_inference/models/application_base.py:552: UserWarning: Found torch.float32 weights in checkpoint: embed_tokens.weight. Will convert to torch.bfloat16
  warnings.warn(
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed_inference/models/application_base.py:552: UserWarning: Found torch.float32 weights in checkpoint: layers.0.self_attn.q_proj.weight. Will convert to torch.bfloat16
  warnings.warn(
/opt/aws_neuronx_venv_pytorch_2_7/lib/python3.10/site-packages/neuronx_distributed_inference/models/application_base.py:552: UserWarning: Found torch.float32 weights in checkpoint: layers.0.self_attn.k_proj.weight. Will convert to torch.bfloa

2025-Oct-23 18:44:49.0575 3258171:3259075 [6] int nccl_net_ofi_create_plugin(nccl_net_ofi_plugin_t**):213 CCOM WARN NET/OFI Failed to initialize sendrecv protocol
2025-Oct-23 18:44:49.0585 3258171:3259075 [6] int nccl_net_ofi_create_plugin(nccl_net_ofi_plugin_t**):354 CCOM WARN NET/OFI aws-ofi-nccl initialization failed
2025-Oct-23 18:44:49.0595 3258171:3259075 [6] ncclResult_t nccl_net_ofi_init_no_atexit_fini_v6(ncclDebugLogger_t):183 CCOM WARN NET/OFI Initializing plugin failed
2025-Oct-23 18:44:49.0605 3258171:3259075 [6] net_plugin.cc:97 CCOM WARN OFI plugin initNet() failed is EFA enabled?


INFO:Neuron:Warmup completed in 0.6640217304229736 seconds.


## Process Dataset and Generate Logits

Process each line in the dataset through the teacher model:
1. Format input as a conversation
2. Generate output with logits
3. Extract finite logits (filter out -inf values)
4. Save results with prompt, generated text, and token logits

In [9]:
results = []
with open(dataset_file, 'r') as f:
    for line in f:
        if line.strip():
            try:
                input_text = create_conversation(line.strip())
                formatted_chat = tokenizer.apply_chat_template(
                    input_text,
                    tokenize=False,
                    add_generation_prompt=True,
                    enable_thinking=False
                )
                inputs = tokenizer(formatted_chat, padding=True, return_tensors="pt")
                generation_model = HuggingFaceGenerationAdapter(model)
                outputs = generation_model.generate(
                    inputs.input_ids,
                    generation_config=generation_config,
                    attention_mask=inputs.attention_mask,
                    max_length=model.config.neuron_config.max_length,
                    return_dict_in_generate=True,
                    output_scores=True,
                    output_logits=True
                )
                
                print(outputs)
                generated_tokens = outputs.sequences[0]
                token_logits = outputs.scores
                generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True, clean_up_tokenization_spaces=False)
                print(generated_text)
                
                token_logits_list = []
                for logits in token_logits:
                    finite_mask = torch.isfinite(logits[0])
                    finite_indices = torch.nonzero(finite_mask).squeeze().tolist()
                    finite_logits = logits[0][finite_mask]
                    token_info = {
                        'indices': finite_indices,
                        'logits': finite_logits.tolist()
                    }
                    token_logits_list.append(token_info)
                
                print(token_logits_list)
                results.append({
                    'prompt': line.strip(),
                    'response': {
                        'generated_text': generated_text,
                        'token_logits': token_logits_list
                    }
                })
            except Exception as e:
                print(f"Error processing prompt: {line[:50]}...")
                print(f"Error message: {str(e)}")
                results.append({
                    'prompt': line.strip(),
                    'error': str(e)
                })

HuggingFaceGenerationAdapter has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
HuggingFaceGenerationAdapter has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit fr

Error processing prompt: The service at this restaurant exceeded all my exp...
Error message: 'super' object has no attribute 'generate'
Error processing prompt: I can't believe how rude the staff was today.
...
Error message: 'super' object has no attribute 'generate'
Error processing prompt: The weather is 72 degrees with partial clouds.
...
Error message: 'super' object has no attribute 'generate'
Error processing prompt: My flight was delayed for the third time this week...
Error message: 'super' object has no attribute 'generate'
Error processing prompt: The package arrived on schedule as expected.
...
Error message: 'super' object has no attribute 'generate'
Error processing prompt: This phone's battery life is absolutely amazing!
...
Error message: 'super' object has no attribute 'generate'
Error processing prompt: The store will be closed from 2PM to 4PM.
...
Error message: 'super' object has no attribute 'generate'
Error processing prompt: I deeply regret purchasing this defec

## Save Results

Write the generated logits and responses to a JSON file for use in distillation training.

In [ ]:
with open(output_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Processing complete! Processed {len(results)} prompts. Results written to {output_file}")